## Theoretical solution
---

The problem can be stated like this:

We have $n$ subjects that produce and consume energy we want to distribute excess energy to those subjects who produced less than they need. We have 5 steps to distribute all the excess energy. We need to come up  with allocation key, here we can imagine vector $p$ satisfying $\sum_{i=1}^{n} p_{i} = 1$ and $p_{i}  \geq 0$ $\forall i \in \{1, ..., n\}$. Vector $p$ represents proportion of excess energy for each subject. *The question is how to determine p such that leftover energy will be minimized?*

Variables in our model:

- $d_i$       -> demand of $i^{th}$ subject
- $p_{i}^{t}$ -> proportion of leftover energy of $i^{th}$ subject in $t^{th}$ iteration
- $r_i$       -> remaining energy to satisfy $i^{th}$ subject
- $E_{t}$     -> Remaining energy

Because we cannot overfill the subject, so update of $r$ will look like this:

$r_{i}(t) = max\{0, r_{i}(t-1) - E_{i}p_{i}^{(t)}\}$

Goal of our model should be to determine sequence $\{p^{(t)}\}_{t=1}^{5}$ so that we minimize $E$. Firstly we will consider basic update -> fill the biggest holes:

$\large p_{i}^{t} = \frac{r_{i}(t-1)}{\sum_{j: r_{j}(t-1) > 0} r_{j}(t-1)}$. 

And we want to minimize:

$Leftover~energy~=~\sum_{i=1}^{n}max\{0, d_i - \sum_{t=1}^{5}E_{t}p_{i}^{(t)}\}$

This approach will be tested against fixed rate algorithm. This algorithm has $p$ fixed and is given by proportion of yearly consumption.

### Modification to models
---

#### Adding priorities

We can add vector of priorities, which would determine who would get bigger piece of the excess energy.

__Example on fixe rate algorithm__:

$Y_i$ -> yearly proportion of energy consumption of $i^{th}$ subject.
$R_i$ -> priority of $i^{th}$ subject.

$p = \frac{Y_i * R_i}{\sum_{i=1}^{n}Y_i * R_i}$

#### Adding battery

We can also add another subject Battery, where we can store excess energy for later.



In [67]:
import numpy as np
import pandas as pd
import time


In [68]:
excess  = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/excess.csv')
deficit = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/deficit.csv')
yearly_cons = pd.read_csv('/home/miro/Bachelor/BT/Analysis/data/outputs/yearly_consumption.csv')

excess['timestamp'] = pd.to_datetime(excess['timestamp'])
excess.set_index('timestamp', inplace=True)
deficit['timestamp'] = pd.to_datetime(deficit['timestamp'])
deficit.set_index('timestamp', inplace=True)

deficit = deficit.astype(float)
excess = excess.astype(float)

In [69]:
excess.describe()

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
count,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.0,35036.0,35036.0,35036.0,35036.000000
mean,0.006574,0.000287,0.007803,0.003103,0.001201,0.003194,0.000719,0.009478,0.000774,0.0,0.0,0.0,0.0,0.007638
std,0.012364,0.000900,0.013330,0.005759,0.002073,0.005759,0.001341,0.019287,0.001924,0.0,0.0,0.0,0.0,0.012950
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
75%,0.006756,0.000000,0.009632,0.003159,0.001639,0.003699,0.000789,0.007602,0.000000,0.0,0.0,0.0,0.0,0.009404
max,0.056772,0.006434,0.057481,0.024997,0.008946,0.025666,0.006022,0.095631,0.011594,0.0,0.0,0.0,0.0,0.056154


In [70]:
deficit.describe()

,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
count,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000,35036.000000
mean,0.001473,0.002241,0.000284,0.000659,0.000146,0.000327,0.000452,0.011466,0.002551,0.004472,0.000725,0.000067,0.032603,0.007638
std,0.002225,0.001602,0.000281,0.000672,0.000244,0.000596,0.000688,0.015986,0.001957,0.005904,0.000424,0.000044,0.009613,0.012950
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000053,0.000000,0.009989,0.000000
25%,0.000000,0.001314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000979,0.000178,0.000415,0.000031,0.024657,0.000000
50%,0.000785,0.002117,0.000326,0.000620,0.000069,0.000118,0.000018,0.003615,0.002265,0.000772,0.000620,0.000062,0.029568,0.000000
75%,0.001226,0.003002,0.000525,0.001078,0.000131,0.000271,0.000839,0.018401,0.004523,0.009089,0.000970,0.000097,0.042373,0.009404
max,0.018844,0.010554,0.001226,0.005282,0.002349,0.007901,0.004165,0.066221,0.008953,0.027600,0.002253,0.000258,0.059035,0.056154


In [71]:
def waterfall_method(deficit, excess, steps):
    """
    Waterfall method for distributing excess energy to buildings with energy deficits.

    Parameters:
    - deficit: DataFrame where each row represents a timestamp and columns represent energy deficits per building.
    - excess: DataFrame where each row represents a timestamp and columns represent available excess energy per building.
    - steps: int, number of distribution steps given by methodics.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    for t in range(steps):
        active_mask = remaining_deficit_df > 0
        total_deficit_per_time = remaining_deficit_df.where(active_mask).sum(axis=1)

        nonzero_mask = total_deficit_per_time > 0
        p = pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns)
        
        p.loc[nonzero_mask] = remaining_deficit_df.loc[nonzero_mask].div(total_deficit_per_time[nonzero_mask], axis=0)
        allocated_energy = excess * p
        allocation_dict[t] = allocated_energy

        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


def fixed_rate_method(deficit, excess, yearly_cons, steps):
    """
    Distributes excess energy based on a fixed rate derived from yearly energy consumption.

    Parameters:
    - deficit: DataFrame where rows represent timestamps and columns represent energy deficits per building.
    - excess: DataFrame where rows represent timestamps and columns represent available excess energy per building.
    - yearly_cons: DataFrame with columns ['Column', 'YY_cons'], mapping buildings to yearly energy consumption.
    - steps: int, number of distribution steps.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    yearly_cons_dict = yearly_cons.set_index('Column')['Y_cons'].to_dict()
    proportions = pd.Series(yearly_cons_dict).reindex(deficit.columns, fill_value=0)
    proportions /= proportions.sum()

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    for t in range(steps):
        allocated_energy = excess.mul(proportions, axis=1)
        allocation_dict[t] = allocated_energy

        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


def waterfall_method_priority(deficit, excess, priority_df, steps):
    """
    Waterfall method for distributing excess energy based on energy deficits and building priority.

    Parameters:
    - deficit: DataFrame where each row represents a timestamp and columns represent energy deficits per building.
    - excess: DataFrame where each row represents a timestamp and columns represent available excess energy per building.
    - priority_df: DataFrame with 'Column' and 'Priority' indicating priority levels for each building.
    - steps: int, number of distribution steps.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    priority_dict = priority_df.set_index('Column')['Priority'].to_dict()
    priorities = pd.Series(priority_dict).reindex(deficit.columns, fill_value=1)
    priorities /= priorities.sum()

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    for t in range(steps):
        active_mask = remaining_deficit_df > 0
        total_deficit_per_time = remaining_deficit_df.where(active_mask).sum(axis=1)

        nonzero_mask = total_deficit_per_time > 0
        p = pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns)

        weighted_deficit = remaining_deficit_df.mul(priorities, axis=1)
        weighted_total = weighted_deficit.where(active_mask).sum(axis=1)

        p.loc[nonzero_mask] = weighted_deficit.loc[nonzero_mask].div(weighted_total[nonzero_mask], axis=0)
        allocated_energy = excess * p
        allocation_dict[t] = allocated_energy

        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


def fixed_rate_method_priority(deficit, excess, yearly_cons, priority_df, steps):
    """
    Distributes excess energy based on a fixed rate derived from yearly energy consumption,
    with added prioritization.

    Parameters:
    - deficit: DataFrame where rows represent timestamps and columns represent energy deficits per building.
    - excess: DataFrame where rows represent timestamps and columns represent available excess energy per building.
    - yearly_cons: DataFrame with columns ['Column', 'Y_cons'], mapping buildings to yearly energy consumption.
    - priority_df: DataFrame with 'Column' and 'Priority' indicating priority levels for each building.
    - steps: int, number of distribution steps.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    yearly_cons_dict = yearly_cons.set_index('Column')['Y_cons'].to_dict()
    yearly_proportions = pd.Series(yearly_cons_dict).reindex(deficit.columns, fill_value=0)
    yearly_proportions /= yearly_proportions.sum()

    priority_dict = priority_df.set_index('Column')['Priority'].to_dict()
    priorities = pd.Series(priority_dict).reindex(deficit.columns, fill_value=1)
    priorities /= priorities.sum()

    adjusted_weights = yearly_proportions * priorities
    adjusted_weights /= adjusted_weights.sum()

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    for t in range(steps):
        allocated_energy = excess.mul(adjusted_weights, axis=1)
        allocation_dict[t] = allocated_energy

        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


def waterfall_method_with_battery(deficit, excess, battery_capacity, efficiency, steps):
    """
    Waterfall method for distributing excess energy to buildings with energy deficits,
    with an added battery model for energy storage.

    Parameters:
    - deficit: DataFrame where each row represents a timestamp and columns represent energy deficits per building.
    - excess: DataFrame where each row represents a timestamp and columns represent available excess energy per building.
    - battery_capacity: float, the maximum capacity of the battery in energy units.
    - efficiency: float, the efficiency of the battery (default is 1.0 for 100% efficiency).
    - steps: int, number of distribution steps.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    - battery_state: List of battery state of charge at each step.
    """

    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    # Initialize battery state (SoC) and storage
    battery_soc = [0.0] * steps  # Battery state of charge (in energy units)
    battery_storage = 0.0  # Battery starts empty

    for t in range(steps):
        active_mask = remaining_deficit_df > 0
        total_deficit_per_time = remaining_deficit_df.where(active_mask).sum(axis=1)

        nonzero_mask = total_deficit_per_time > 0
        p = pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns)
        
        p.loc[nonzero_mask] = remaining_deficit_df.loc[nonzero_mask].div(total_deficit_per_time[nonzero_mask], axis=0)
        allocated_energy = excess * p
        
        # Now consider the battery model:
        # First, we check if there is excess energy and the battery can store it
        excess_energy_for_battery = excess.sum(axis=1) - allocated_energy.sum(axis=1)  # Excess after allocation
        excess_energy_for_battery = np.clip(excess_energy_for_battery, 0, battery_capacity - battery_storage)

        # Charge the battery
        battery_storage += efficiency * excess_energy_for_battery.sum()  # Efficiency of charging
        battery_storage = np.clip(battery_storage, 0, battery_capacity)  # Ensure the battery doesn't exceed capacity

        # Then, if there is a deficit, the battery can supply energy
        energy_needed_from_battery = remaining_deficit_df.sum(axis=1) - allocated_energy.sum(axis=1)
        energy_needed_from_battery = np.clip(energy_needed_from_battery, 0, battery_storage)

        # Convert to numpy array to apply broadcasting safely
        allocated_energy += np.expand_dims(energy_needed_from_battery, axis=1)  # Add battery energy to allocated energy
        battery_storage -= energy_needed_from_battery.sum()  # Decrease the battery storage

        # Record the battery state of charge
        battery_soc[t] = battery_storage

        # Update remaining deficit
        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

        allocation_dict[t] = allocated_energy

    return allocation_dict, remaining_deficit_df #, battery_soc

def fixed_rate_method_with_battery(deficit, excess, yearly_cons, battery_capacity, efficiency, steps):
    """
    Fixed rate method for distributing excess energy to buildings with energy deficits, 
    with an added battery model for energy storage.

    Parameters:
    - deficit: DataFrame where each row represents a timestamp and columns represent energy deficits per building.
    - excess: DataFrame where each row represents a timestamp and columns represent available excess energy per building.
    - yearly_cons: DataFrame mapping consumers to yearly consumption (used for fixed rate allocation).
    - priority_df: DataFrame defining priority of each producer for each consumer (optional).
    - battery_capacity: float, the maximum capacity of the battery in energy units (default is 1000).
    - efficiency: float, the efficiency of the battery (default is 1.0 for 100% efficiency).
    - steps: int, number of distribution steps.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values,
      showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    - battery_state: List of battery state of charge at each step.
    """
    
    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    # Initialize battery state (SoC) and storage
    battery_soc = [0.0] * steps  # Battery state of charge (in energy units)
    battery_storage = 0.0  # Battery starts empty

    # Calculate fixed rate distribution based on yearly consumption
    total_yearly_consumption = yearly_cons['Y_cons'].sum()
    fixed_rate = yearly_cons['Y_cons'].div(total_yearly_consumption)

    for t in range(steps):
        # Step 1: Allocate energy based on the fixed rate
        active_mask = remaining_deficit_df > 0
        total_deficit_per_time = remaining_deficit_df.where(active_mask).sum(axis=1)

        nonzero_mask = total_deficit_per_time > 0
        p = pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns)
        
        p.loc[nonzero_mask] = remaining_deficit_df.loc[nonzero_mask].div(total_deficit_per_time[nonzero_mask], axis=0)

        # Fixed rate allocation
        allocated_energy = fixed_rate * excess
        allocated_energy = allocated_energy.clip(upper=remaining_deficit_df)

        # Step 2: Battery Charging (if there's excess energy available)
        excess_energy_for_battery = excess.sum(axis=1) - allocated_energy.sum(axis=1)  # Excess after allocation
        excess_energy_for_battery = np.clip(excess_energy_for_battery, 0, battery_capacity - battery_storage)

        # Charge the battery
        battery_storage += efficiency * excess_energy_for_battery.sum()  # Efficiency of charging
        battery_storage = np.clip(battery_storage, 0, battery_capacity)  # Ensure battery doesn't exceed capacity

        # Step 3: Battery Discharge (if there's a remaining deficit)
        energy_needed_from_battery = remaining_deficit_df.sum(axis=1) - allocated_energy.sum(axis=1)
        energy_needed_from_battery = np.clip(energy_needed_from_battery, 0, battery_storage)

        # Update the allocated energy by using battery storage
        allocated_energy += np.expand_dims(energy_needed_from_battery, axis=1)  # Add battery energy to allocated energy
        battery_storage -= energy_needed_from_battery.sum()  # Decrease the battery storage

        # Step 4: Record battery state of charge
        battery_soc[t] = battery_storage

        # Update remaining deficit
        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

        allocation_dict[t] = allocated_energy

    return allocation_dict, remaining_deficit_df #, battery_soc

def create_priority_vector(yearly_cons):
    priority_df = yearly_cons['Column'].to_frame()
    vec = yearly_cons['Y_cons']
    if vec.ndim > 1:
        vec = vec.sum(axis=0)
    max_cons = vec.max()
    priority_vector = vec / max_cons
    priority_df['Priority'] = priority_vector
    return priority_df



In [72]:
def benchmark_methods(deficit, excess, yearly_cons=None, priority_df=None, methods=None, steps=5, battery_capacity=None, efficiency=None):
    """
    Benchmarks different energy distribution methods by timing their execution 
    and evaluating remaining deficit.

    Parameters:
    - deficit: DataFrame where rows are timestamps and columns are consumers.
    - excess: DataFrame where rows are timestamps and columns are producers.
    - yearly_cons: DataFrame mapping consumers to yearly consumption (if needed).
    - priority_df: DataFrame defining priority of each producer for each consumer.
    - methods: Dictionary with function names as keys and function objects as values.
               Example: {'Waterfall': waterfall_method, 'Fixed Rate': fixed_rate_method}
    - steps: int, number of distribution steps.

    Returns:
    - summary_df: DataFrame summarizing execution time and remaining deficit for each method.
    - results: Dictionary storing detailed allocation results for each method.
    """

    if methods is None:
        raise ValueError("Please provide a dictionary of methods to benchmark.")

    results = {}
    summary = []

    for method_name, method_func in methods.items():
        start_time = time.time()

        # Determine which parameters are needed and call method accordingly
        if 'priority_df' in method_func.__code__.co_varnames and priority_df is not None:
            if 'yearly_cons' in method_func.__code__.co_varnames and yearly_cons is not None:
                allocation, remaining_deficit = method_func(deficit, excess, yearly_cons, priority_df, steps=steps)
            else:
                allocation, remaining_deficit = method_func(deficit, excess, priority_df, steps=steps)
        elif 'yearly_cons' in method_func.__code__.co_varnames and yearly_cons is not None:
            if 'battery_capacity' in method_func.__code__.co_varnames and battery_capacity is not None:
                allocation, remaining_deficit = method_func(deficit, excess, yearly_cons, battery_capacity, efficiency, steps=steps)
            else:
                allocation, remaining_deficit = method_func(deficit, excess, yearly_cons, steps=steps)
        elif 'battery_capacity' in method_func.__code__.co_varnames and battery_capacity is not None:
                allocation, remaining_deficit = method_func(deficit, excess, battery_capacity, efficiency=efficiency, steps=steps)
        else:
            allocation, remaining_deficit = method_func(deficit, excess, steps=steps)

        execution_time = time.time() - start_time
        total_remaining_deficit = remaining_deficit.sum().sum()

        # Store results
        results[method_name] = {'allocation': allocation, 'remaining_deficit': remaining_deficit}

        # Append summary stats
        summary.append({
            'Method': method_name,
            'Execution Time (s)': execution_time,
            'Total Remaining Deficit': total_remaining_deficit,
            'Total Deficit': deficit.sum().sum(),
            'Total Excess': excess.sum().sum(),
            'Total Number of Consumers': deficit.shape[1],
            'Total Number of Timestamps': deficit.shape[0]
        })

        print(f"{method_name} - Execution Time: {execution_time:.4f} seconds, Remaining Deficit: {total_remaining_deficit:.2f}")

    # Convert summary to DataFrame
    summary_df = pd.DataFrame(summary)

    return summary_df, results


In [73]:

# Define baterry capacity based on average excess energy
battery_capacity = excess.mean().mean()

In [74]:
battery_capacity

np.float64(0.0029120834494410835)

In [75]:
priority_df = create_priority_vector(yearly_cons) 

methods_to_test = {
    'Waterfall': waterfall_method,
    'Fixed Rate': fixed_rate_method,
    'Waterfall Priority': waterfall_method_priority,
    'Fixed Rate Priority': fixed_rate_method_priority,
    'Waterfall with Battery': waterfall_method_with_battery,
    'Fixed Rate with Battery': fixed_rate_method_with_battery
}
summary, results = benchmark_methods(deficit, excess, 
                                     yearly_cons=yearly_cons, 
                                     priority_df=priority_df, 
                                     methods=methods_to_test, 
                                     steps=5,
                                     battery_capacity=battery_capacity,
                                     efficiency=0.8)

summary


Waterfall - Execution Time: 0.2914 seconds, Remaining Deficit: 2047.45
Fixed Rate - Execution Time: 0.0578 seconds, Remaining Deficit: 2234.06
Waterfall Priority - Execution Time: 0.3019 seconds, Remaining Deficit: 2213.57
Fixed Rate Priority - Execution Time: 0.0469 seconds, Remaining Deficit: 2274.83
Waterfall with Battery - Execution Time: 0.3904 seconds, Remaining Deficit: 834.41
Fixed Rate with Battery - Execution Time: 0.6183 seconds, Remaining Deficit: 0.00


,Method,Execution Time (s),Total Remaining Deficit,Total Deficit,Total Excess,Total Number of Consumers,Total Number of Timestamps
0,Waterfall,0.291438,2047.445311,2280.88966,1428.38858,14,35036
1,Fixed Rate,0.057781,2234.063266,2280.88966,1428.38858,14,35036
2,Waterfall Priority,0.301904,2213.567180,2280.88966,1428.38858,14,35036
3,Fixed Rate Priority,0.046925,2274.834240,2280.88966,1428.38858,14,35036
4,Waterfall with Battery,0.390386,834.406218,2280.88966,1428.38858,14,35036
5,Fixed Rate with Battery,0.618271,0.000000,2280.88966,1428.38858,14,35036


In [76]:
# Have to correct code for Fixed Rate with Battery method because it gives weird results...

In [77]:
### CONCISE MODELS

def waterfall_method(deficit, excess, steps, with_battery=False, battery_capacity=None, efficiency=None, with_priority=False, priority_df=None):
    """
    Waterfall method for distributing excess energy to buildings with energy deficits,
    optionally with priority and/or battery.

    Parameters:
    - deficit: DataFrame where each row represents a timestamp and columns represent energy deficits per building.
    - excess: DataFrame where each row represents a timestamp and columns represent available excess energy per building.
    - steps: int, number of distribution steps.
    - with_battery: boolean, if True, includes battery storage logic.
    - battery_capacity: float, the maximum capacity of the battery if with_battery is True.
    - efficiency: float, the efficiency of the battery if with_battery is True.
    - with_priority: boolean, if True, includes priority logic.
    - priority_df: DataFrame with columns ['Column', 'Priority'], if with_priority is True.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values, showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """
    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    # Apply priority logic if needed
    if with_priority:
        priority_dict = priority_df.set_index('Column')['Priority'].to_dict()
        priorities = pd.Series(priority_dict).reindex(deficit.columns, fill_value=1)
        priorities /= priorities.sum()
    else:
        priorities = pd.Series(1.0, index=deficit.columns)  # Equal priority

    for t in range(steps):
        active_mask = remaining_deficit_df > 0
        total_deficit_per_time = remaining_deficit_df.where(active_mask).sum(axis=1)

        nonzero_mask = total_deficit_per_time > 0
        p = pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns)
        weighted_deficit = remaining_deficit_df.mul(priorities, axis=1)
        weighted_total = weighted_deficit.where(active_mask).sum(axis=1)
        
        p.loc[nonzero_mask] = weighted_deficit.loc[nonzero_mask].div(weighted_total[nonzero_mask], axis=0)
        allocated_energy = excess * p

        # Battery logic if enabled
        if with_battery:
            # First, check if there is excess energy and the battery can store it
            excess_energy_for_battery = excess.sum(axis=1) - allocated_energy.sum(axis=1)
            excess_energy_for_battery = np.clip(excess_energy_for_battery, 0, battery_capacity)

            # Charge the battery
            battery_storage += efficiency * excess_energy_for_battery.sum()
            battery_storage = np.clip(battery_storage, 0, battery_capacity)

            # Then, check if there is a deficit and battery can supply energy
            energy_needed_from_battery = remaining_deficit_df.sum(axis=1) - allocated_energy.sum(axis=1)
            energy_needed_from_battery = np.clip(energy_needed_from_battery, 0, battery_storage)

            # Add energy from battery to allocated energy
            allocated_energy += np.expand_dims(energy_needed_from_battery, axis=1)
            battery_storage -= energy_needed_from_battery.sum()

        allocation_dict[t] = allocated_energy
        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


def fixed_rate_method(deficit, excess, yearly_cons, steps, with_battery=False, battery_capacity=None, efficiency=None, with_priority=False, priority_df=None):
    """
    Fixed rate method for distributing excess energy to buildings with energy deficits,
    optionally with priority and/or battery.

    Parameters:
    - deficit: DataFrame where rows represent timestamps and columns represent energy deficits per building.
    - excess: DataFrame where rows represent timestamps and columns represent available excess energy per building.
    - yearly_cons: DataFrame with columns ['Column', 'Y_cons'], mapping buildings to yearly energy consumption.
    - steps: int, number of distribution steps.
    - with_battery: boolean, if True, includes battery storage logic.
    - battery_capacity: float, the maximum capacity of the battery if with_battery is True.
    - efficiency: float, the efficiency of the battery if with_battery is True.
    - with_priority: boolean, if True, includes priority logic.
    - priority_df: DataFrame with columns ['Column', 'Priority'], if with_priority is True.

    Returns:
    - allocation_dict: Dictionary with timestamps as keys and DataFrames as values, showing how energy was distributed at each step.
    - remaining_deficit_df: DataFrame showing remaining energy needs after all steps.
    """
    common_timestamps = deficit.index.intersection(excess.index)
    deficit = deficit.loc[common_timestamps]
    excess = excess.loc[common_timestamps]

    # Calculate proportions based on yearly consumption
    yearly_cons_dict = yearly_cons.set_index('Column')['Y_cons'].to_dict()
    proportions = pd.Series(yearly_cons_dict).reindex(deficit.columns, fill_value=0)
    proportions /= proportions.sum()

    allocation_dict = {t: pd.DataFrame(0.0, index=deficit.index, columns=deficit.columns) for t in range(steps)}
    remaining_deficit_df = deficit.copy()

    # Apply priority logic if needed
    if with_priority:
        priority_dict = priority_df.set_index('Column')['Priority'].to_dict()
        priorities = pd.Series(priority_dict).reindex(deficit.columns, fill_value=1)
        priorities /= priorities.sum()
    else:
        priorities = pd.Series(1.0, index=deficit.columns)  # Equal priority

    for t in range(steps):
        # Allocate energy based on fixed proportions
        allocated_energy = excess.mul(proportions, axis=1)

        # Battery logic if enabled
        if with_battery:
            # First, check if there is excess energy and the battery can store it
            excess_energy_for_battery = excess.sum(axis=1) - allocated_energy.sum(axis=1)
            excess_energy_for_battery = np.clip(excess_energy_for_battery, 0, battery_capacity)

            # Charge the battery
            battery_storage += efficiency * excess_energy_for_battery.sum()
            battery_storage = np.clip(battery_storage, 0, battery_capacity)

            # Then, check if there is a deficit and battery can supply energy
            energy_needed_from_battery = remaining_deficit_df.sum(axis=1) - allocated_energy.sum(axis=1)
            energy_needed_from_battery = np.clip(energy_needed_from_battery, 0, battery_storage)

            # Add energy from battery to allocated energy
            allocated_energy += np.expand_dims(energy_needed_from_battery, axis=1)
            battery_storage -= energy_needed_from_battery.sum()

        allocation_dict[t] = allocated_energy
        remaining_deficit_df -= allocated_energy
        remaining_deficit_df = remaining_deficit_df.clip(lower=0)

    return allocation_dict, remaining_deficit_df


